In [1]:
import bz2
from tqdm import tqdm
import re
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords

### Extract bz2 files

In [2]:

path_train='./data/train.ft.txt'

with bz2.open('./data/train.ft.txt.bz2', 'rt', encoding='utf-8') as compressed_file, open(path_train, 'w', encoding='utf-8') as output_file:
    for line in compressed_file:
        output_file.write(line)
    print('Train data export successful.')
    
path_test='./data/test.ft.txt'

with bz2.open('./data/test.ft.txt.bz2', 'rt', encoding='utf-8') as compressed_file, open(path_test, 'w', encoding='utf-8') as output_file:
    for line in compressed_file:
        output_file.write(line)
    print('Test data export successful.')

Train data export successful.
Test data export successful.


### Check txt files

In [3]:
# Specify the number of lines you want to display
num_lines = 5  # Change this number as needed

# Open the file and read the first 'num_lines' lines
with open(path_train, 'r') as file:
    for i, line in enumerate(file):
        if i < num_lines:
            print(line.strip())  # Strip removes trailing newline characters
        else:
            break

__label__2 Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^
__label__2 The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny.
__label__2 Amazing!: This soundtrack is m

In [4]:
train,test,train_label,test_label=[],[],[],[]
with open(path_train, 'r', encoding='utf-8') as file:
    lines = file.readlines()
for line in tqdm(lines):
    train.append(line.split('__label__')[1][1:])
    train_label.append(line.split('__label__')[1][0])
with open(path_test, 'r', encoding='utf-8') as file:
    lines = file.readlines()
for line in tqdm(lines):
    test.append(line.split('__label__')[1][1:])
    test_label.append(line.split('__label__')[1][0])

100%|██████████| 400000/400000 [00:00<00:00, 842683.21it/s]


In [5]:
# select a random sample for faster processing
import random
seed = 123

# Randomly select elements from the train list
random.seed(seed)
train_indices = random.sample(range(len(train)), 4000)
train = [train[i] for i in train_indices]
train_label = [train_label[i] for i in train_indices]

# Randomly select elements from the test list
random.seed(seed)
test_indices = random.sample(range(len(test)), 200)
test = [test[i] for i in test_indices]
test_label = [test_label[i] for i in test_indices]

In [6]:
import pickle

# Save to pickle files
with open('data/train.pkl', 'wb') as f:
    pickle.dump(train, f)
with open('data/train_label.pkl', 'wb') as f:
    pickle.dump(train_label, f)
with open('data/test.pkl', 'wb') as f:
    pickle.dump(test, f)
with open('data/test_label.pkl', 'wb') as f:
    pickle.dump(test_label, f)

In [7]:
# Load from pickle files
with open('data/train.pkl', 'rb') as f:
    train = pickle.load(f)
with open('data/train_label.pkl', 'rb') as f:
    train_label = pickle.load(f)
with open('data/test.pkl', 'rb') as f:
    test = pickle.load(f)
with open('data/test_label.pkl', 'rb') as f:
    test_label = pickle.load(f)

In [8]:
def clean_text(text):
    # Remove non-alphanumeric characters and extra whitespace
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert multiple whitespace characters to a single space
    text = re.sub(r'\s+', ' ', text)
    # Convert the text to lowercase
    text = text.lower()
    return text

In [9]:
print('Train Length',len(train))
print('Train Label Length',len(train_label))
print('Test Length',len(test))
print('Test Label Length',len(test_label))

Train Length 4000
Train Label Length 4000
Test Length 200
Test Label Length 200


In [10]:
train_label[0],train[0]

('2',
 ' A Different View of Blink 182: If you bought this CD after buying Enema of the state, like I did, youll be suprised at how much they have changed. This Cd has great songs and gives you another side of Blink-182. I like it.\n')

In [11]:
train_label[0],clean_text(train[0])

('2',
 ' a different view of blink if you bought this cd after buying enema of the state like i did youll be suprised at how much they have changed this cd has great songs and gives you another side of blink i like it ')

In [12]:
train=pd.DataFrame(train)[0].apply(clean_text)
test=pd.DataFrame(test)[0].apply(clean_text)
train.head()

0     a different view of blink if you bought this ...
1     a holiday classic this was one of my favorite...
2     np mistake do not buy this productthis price ...
3     you may not get what you ordered i ordered yi...
4     a great game that might have been the temple ...
Name: 0, dtype: object

In [13]:
# Load stopwords into a global variable
# stopwords = set(nltk.corpus.stopwords.words('english'))

# # function to tokenize and remove stopwords from a single text
# def remove_stopwords(series):
#     return series.apply(lambda text: ' '.join(word for word in nltk.word_tokenize(text) if word.lower() not in stopwords))

In [14]:
# tokenize and remove stopwords in parallel
# def parallelize_tokenizer(df, func, n_cores=4):
#     df_split = np.array_split(df, n_cores)
#     pool = Parallel(n_jobs=n_cores)
#     df = pd.concat(pool(delayed(func)(i) for i in df_split))
#     return df

# num_cores = joblib.cpu_count()
# train_token = parallelize_tokenizer(train, remove_stopwords,num_cores)
# test_token = parallelize_tokenizer(test, remove_stopwords,num_cores)


In [15]:
from transformers import BertTokenizer

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and pad sequences
def tokenize_and_pad(text, max_len):
    tokenized = tokenizer.encode_plus(
        text, 
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )
    return tokenized["input_ids"], tokenized["attention_mask"]

max_len = 512

# Tokenize the sequences
train_tokenized = [tokenize_and_pad(text, max_len) for text in train]
test_tokenized = [tokenize_and_pad(text, max_len) for text in test]


c:\Users\leocb\OneDrive\Documentos\Projects\ReviewsAnalysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Save to pytorch files:

In [16]:
train_label_int = [int(t)-1 for t in train_label]
test_label_int = [int(t)-1 for t in test_label]

In [17]:
import torch

# Convert train_labels to a tensor
train_labels_tensor = torch.tensor(train_label_int)
test_labels_tensor = torch.tensor(test_label_int)

# Save train_tokenized and train_labels_tensor
torch.save((train_tokenized, train_labels_tensor), 'data/train_data.pt')
torch.save((test_tokenized, test_labels_tensor), 'data/test_data.pt')

# Modelling

In [18]:
import torch
from transformers import BertForSequenceClassification

# Load the BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Load the saved data
train_data = torch.load('data/train_data.pt')
test_data = torch.load('data/test_data.pt')

# Separate the tokenized data and labels
train_tokenized, train_labels_tensor = train_data
test_tokenized, test_labels_tensor = test_data

# Convert the tokenized data into tensors
train_inputs = torch.stack([item[0] for item in train_tokenized]).squeeze()
train_masks = torch.stack([item[1] for item in train_tokenized]).squeeze()
test_inputs = torch.stack([item[0] for item in test_tokenized]).squeeze()
test_masks = torch.stack([item[1] for item in test_tokenized]).squeeze()

# Move everything to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
train_inputs = train_inputs.to(device)
train_masks = train_masks.to(device)
train_labels_tensor = train_labels_tensor.to(device)
test_inputs = test_inputs.to(device)
test_masks = test_masks.to(device)
test_labels_tensor = test_labels_tensor.to(device)

# Predict the labels for the test data
model.eval()
with torch.no_grad():
    outputs = model(test_inputs, attention_mask=test_masks)
    _, predicted_labels = torch.max(outputs.logits, 1)

print(predicted_labels)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])


Save model to pickle:

In [19]:
# Save the model
model.save_pretrained('data/model')

In [20]:
# load the model
model = BertForSequenceClassification.from_pretrained('data/model')

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Move tensors back to CPU for metric calculation
predicted_labels_np = predicted_labels.cpu().numpy()
test_labels_np = test_labels_tensor.cpu().numpy()

# Calculate metrics
accuracy = accuracy_score(test_labels_np, predicted_labels_np)
precision = precision_score(test_labels_np, predicted_labels_np)
recall = recall_score(test_labels_np, predicted_labels_np)
f1 = f1_score(test_labels_np, predicted_labels_np)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Accuracy: 0.535
Precision: 0.0
Recall: 0.0
F1 Score: 0.0


c:\Users\leocb\OneDrive\Documentos\Projects\ReviewsAnalysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
